In [ ]:
import cv2
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Conv2D, Flatten,MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [ ]:
train_dir = '../input/arabic-hwr-ai-pro-intake1/train/'
test_dir = '../input/arabic-hwr-ai-pro-intake1/test/'

files1,files2 = [],[]

files1.extend(glob.glob(train_dir + '*.' + 'png')) 
files1 = sorted(files1)
train_images = np.array([cv2.imread(file,cv2.IMREAD_GRAYSCALE) for file in files1])

train_labels = np.genfromtxt('../input/arabic-hwr-ai-pro-intake1/train.csv',delimiter=',',dtype=int,skip_header=1)[:,1]

files2.extend(glob.glob(test_dir + '*.' + 'png'))
files2 = sorted(files2)
test_images = np.array([cv2.imread(file,cv2.IMREAD_GRAYSCALE) for file in files2])
test_labels = np.genfromtxt('../input/arabic-hwr-ai-pro-intake1/test.csv',delimiter=',',dtype=int,skip_header=1)


In [ ]:
train_images = train_images.reshape(-1, 32, 32, 1)
test_images = test_images.reshape(-1, 32, 32, 1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
train_labels = encoder.fit_transform(train_labels.reshape(-1,1))

In [ ]:
from sklearn.model_selection import train_test_split
train,validation,y_train,y_val = train_test_split(train_images,train_labels, test_size=0.2, random_state=42,shuffle=True,stratify=train_labels)

In [ ]:
train = train/255
validation = validation/255

In [ ]:
model = Sequential()
model.add(Conv2D(64,kernel_size=3,activation='leaky_relu',input_shape=(32,32,1),padding='same'))
model.add(Conv2D(128,kernel_size=3,activation='leaky_relu',padding='same'))
model.add(MaxPool2D(2))
model.add(Dropout(0.3))
model.add(Conv2D(256,kernel_size=3,activation='leaky_relu',padding='same'))
model.add(Dropout(0.4))
model.add(Conv2D(512,kernel_size=3,activation='leaky_relu',padding='same'))
model.add(MaxPool2D(2))
model.add(Dropout(0.4))
model.add(Conv2D(256,kernel_size=3,activation='leaky_relu',padding='same'))
model.add(MaxPool2D(2))
model.add(Dropout(0.4))
model.add(MaxPool2D(2))
model.add(Flatten())
model.add(Dense(400,activation='leaky_relu'))
model.add(Dropout(.3))
model.add(Dense(256,activation='relu'))
model.add(Dropout(.3))
model.add(Dense(28,activation='softmax'))
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5)

training = model.fit(train,y_train ,epochs = 50,batch_size=128,validation_data=(validation,y_val),
                  callbacks=reduce_lr    )

In [ ]:
fig1 = plt.figure()
plt.plot(range(50),training.history['loss'])
plt.plot(range(50),training.history['val_loss'])

plt.ylabel('loss',fontsize=10)
plt.xlabel('epoch',fontsize=10)
plt.legend(['train', 'test'], loc='upper right',fontsize=10)
plt.show()

In [ ]:
accuracy = model.evaluate(train, y_train)[1]
print('trainAccuracy: %.2f' % (accuracy*100)+"%")

accuracy = model.evaluate(validation, y_val)[1]
print('val Accuracy: %.2f' % (accuracy*100)+"%")

In [ ]:
model.save("./model.h5")

In [ ]:
from tensorflow.keras.models import load_model
x = load_model('model.h5')
x.evaluate(train,y_train)[1]

In [ ]:
predicitons = model.predict(test_images/255)
predicitons = np.argmax(predicitons,axis=1)
predicitons

In [ ]:
pred = x.predict(validation)
pred = np.argmax(pred,axis=1)
pred = pred+1
y_val = np.argmax(y_val,axis=1)
y_val = y_val+1

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, pred)
# print(cm)
figure, ax = plot_confusion_matrix(conf_mat = cm,show_normed = True,figsize=(28,28))
plt.ylabel()
plt.show()

In [ ]:
sub = pd.read_csv('../input/arabic-hwr-ai-pro-intake1/sample_submission.csv')
sub['label']=predicitons+1
sub.to_csv("./submission_file",index=False)

In [ ]:
pd.read_csv("./submission_file")